# ASHE Table 7

-----

### Requirements

"Annual Summary of Hours and Earnings"


#### Observations & Dimensions

The `observations` are the numbers in the percentile columns.

The required dimensions are:

* **Geography** - in the `Code` column, one letter followed by 8 digits
* **Percentiles** - 10,20,30, etc
* **Time** - year, 4 digits
* **Gender** - Male, Female, All
* **Working Pattern** - Full time, Part time, All
* **Statistics** - The "topic" of the dataset, i.e "monthly pay net etc", in the filename

-----
    
Notes:

The "statistics" seems pointless because we're looking at one file. In production there are 24 per year per ASHE table.

It's always worth getting the file out of /sources and having a look over.

In [2]:
% cd mock-transformations/

UsageError: Line magic function `%` not found.


In [7]:
from databaker.framework import *
import pandas as pd

tabs = loadxlstabs(r"""./sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls""")

FileNotFoundError: [Errno 2] No such file or directory: './sources/PROV - Work Geography Table 7.1a   Weekly pay - Gross 2018.xls'